In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re

In [2]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()

display(all_projects_source_codes.head())
display(all_projects_bugreports.head())

*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


,filename,unprocessed_code,project
0,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
1,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
2,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
3,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS
4,\gitrepo\src\java\org\apache\commons\collectio...,/*\n * Licensed to the Apache Software Founda...,COLLECTIONS


,fix,text,fixdate,summary,description,project,average_precision
id,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3Map.Entry.setValue() overwrites other Ent...,Flat3Map&amp;apos;s Entry objects will overwri...,COLLECTIONS,0.0
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,ExtendedProperties - field include should be n...,"The field ""include"" in ExtendedProperties is c...",COLLECTIONS,0.0
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,CollectionUtils removeAll is actually retainAll,"The removeAll(Collection collection, Collectio...",COLLECTIONS,0.0
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3Map.remove() does not return the correct ...,final Flat3Map m = new Flat3Map();\n ...,COLLECTIONS,0.0
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,FastTreeMap forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0


#### Removing New Lines

In [3]:
#remove next line characters:
def remove_new_lines(text):
    text = str(text)
    COMBINE_WHITE_SPACE = re.compile(r"(?a:\s+)")
    text = COMBINE_WHITE_SPACE.sub(' ', text)
    return text.replace('*', '').replace('/', '').replace('\\','')
    
# clean up the various white space and remove some *
def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

# clean up the description and summary, they will both be used for the query
def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|U')
    df.description = df.description.apply(remove_new_lines)
    return df

#### Cleaning file paths

In [4]:
# changes file path to be just the filename + extension for source code files
def clean_sc_file(x):
    file = x.split("\\")
    return ''.join(file[-1:])

# changes file path to be just the filename + extension for bug report fixes 
def clean_bug_file(x):
    fixes = []

    for file in x:
        file = file.split(".")
        file = '.'.join(file[-2:])
        fixes.append(file)
    return fixes


def clean_sc_filepath(df):
    df.filename = df.filename.map(clean_sc_file)
    return df


def clean_bug_filepath(df):
    df['fix'] = df['fix'].map(clean_bug_file)
    return df

#### Cleaning Composite Words

In [5]:
#splitting composite words
#splits using camlecase syntax
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df

#### Remove fixes that can't be found

In [6]:
# look through the src data frame to find where the fix is. 
def get_fix_indexes(bug, src):
    fix_list = list()
    for fixes in bug["fix"]:
        fix_sub=list()
        for fix in fixes:
            df = src[src["filename"].str.match(fix)]
            if(df.shape[0] != 0):
                fix_sub.append(df.index[0])
            else:
                fix_sub.append(-1)
        fix_list.append(fix_sub)
    # this is a list of the indexes of the file where the fix was located
    return fix_list

In [115]:
def removeFixesNotFound(bug, src):
    bug["fix_indexes"] = get_fix_indexes(bug, src)
    fixes = bug.fix.tolist()
    fix_indexes = bug.fix_indexes.tolist()
    fixes_return = []
    fixes_indexes_return = []
    numFixes = []
    for i in range(len(fixes)):
        fixes_temp = []
        indexes_temp = []
        numFixes.append(len(fix_indexes[i]))
        for l in range(len(fix_indexes[i])):
            if fix_indexes[i][l] != -1:           
                fixes_temp.append(fixes[i][l])
                indexes_temp.append(fix_indexes[i][l])
        if len(fixes_temp) == 0:
            fixes_return.append(np.nan)
            fixes_indexes_return.append(np.nan)
        else:
            fixes_return.append(fixes_temp)
            fixes_indexes_return.append(indexes_temp)
        
#         print(fixes_return)
#         print(fixes_indexes_return)
    bug['numFixes'] = numFixes
    bug['fix'] = fixes_return
    bug['fix_indexes'] = fixes_indexes_return 
    
    return bug

#### Calling cleaning functions

In [8]:
# clean up the unprocessed code column
def clean_source_df(df):
    # clean up the new lines
    df = clean_new_lines_source_code(df)
    # clean up composite words
    df = clean_composite_source_code(df)
    # clean filepaths
    df = clean_sc_filepath(df)
    return df

# add the summary and description together and clean the data
def clean_combine_bug_df(df):
    # clean up new lines
    df = clean_new_lines_bug_report(df)
    # clean composite words
    df = clean_composite_bug_report(df)
    # clean file path
    df = clean_bug_filepath(df)
    # combine summary and descriptions to create query
    df["query"] = df["summary"] + df["description"]
    return df



### Run Cleaning and Setup Functions

In [116]:
all_projects_bugreports = all_projects_bugreports.dropna(axis=0, subset=['fix'], how='all')

#  get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_combine_bug_df(all_projects_bugreports)


# remove fixes that aren't found
br_df = removeFixesNotFound(br_df, sc_df)
br_df = br_df.dropna(axis=0, subset=['fix','fix_indexes'], how='all')


### Save the clean DFs as pickle files to prevent having to clean them again

In [12]:
sc_df.to_pickle("./Output/cleanSource.pickle")
br_df.to_pickle("./Output/cleanBugs.pickle")

#### Combining stop words, keywords and operators

In [14]:
# add the Java key words to the stop words
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
java_operators = ["+", "-", "*", "/", "%", "+=", "-=", "*=", "/=", "++", "--", "==", "!=", "<", ">", "<=", ">=", ".", "[", "]", "(",")", "!", "~","instanceof", "<<", ">>", ">>>", "&", "^", "|", "&&", "||", "?", ":", "^=", "%=", "<<=", ">>=", ">>>=", "&="]
stop = java_keywords + java_operators
#contains english stop words, java keywords and java operators
STOP_WORDS = ENGLISH_STOP_WORDS.union(stop)

# remove the stem and stop words
# takes in an array of strings returns an array of strings
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()
    return text



## Gensim


### Get a Series with all the source code files and all the bug reports

In [15]:
# create a series with all the source code and all the
sc_df.reset_index(drop=True, inplace=True)
training_src = sc_df.iloc[:, 0:3].copy()
training_src.columns = ['filename', 'query', 'project']

In [101]:
training_bugs = br_df[["query", "project"]].copy()
training_bugs['filename'] = 'bug'
training_bugs.reset_index(drop=True, inplace=True)
training_bugs = training_bugs[['filename', 'query','project']]

In [17]:
# combine the two columns to create a single data frame to train the model
training_data = pd.concat([training_src, training_bugs], ignore_index=True)
training_data['query'] = training_data['query'].map(stem_stop)
training_data

,filename,query,project
0,arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
12140,bug,Undertow H T T P S listener offers no cipher ...,ELY
12141,bug,Missing null check in equals() method of Abst...,ELY
12142,bug,No log messages comming from Elytron - group ...,ELY
12143,bug,Elytron introduces S S L T L S protocol const...,ELY


### Now start training the Gensim model

In [19]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# get the tagged documents for the doc2vec model
training_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(training_data['query']))]
training_docs

# initialize model
# vector_size = dimensionality of the feature vectors (25, 100, 200)
# window = the max distance bweteeen the predicted word and context words "TUNE"
# alpha = the initial learning rate (says that this will drop as training progresses) start at 0.05? 0.025?
# seed = for reproducibility (MAKE SURE IT WORKS) "TUNE"  [says you need 1 worker for reproducibility]
# min_count = ignore all words with a frequency lower than this
# max_vocab size = limit RAM during vocab building every 1 million words needs 1GB of RAM
# workers = number of worker threads used to train. 
# epochs = number of epochs over the corpus (10-20??)

# time the single run
%timeit -n1 -r1 doc_model = Doc2Vec(training_docs, vector_size=25, window=3, alpha=0.05, min_count=1, seed=42, workers=1, epochs=20)

# build vocabulary

# train model



5min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [20]:
%timeit -n1 -r1 doc_model3 = Doc2Vec(training_docs, vector_size=200, window=3, alpha=0.05, min_count=1, seed=42, workers=1, epochs=20)


6min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [105]:
%timeit -n1 -r1 doc_model5 = Doc2Vec(training_docs, vector_size=200, window=3, alpha=0.05, min_count=1, seed=42, workers=6, epochs=20)


3min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Currently working with model3

In [29]:
# print the model vocabular
tst_model = Doc2Vec(training_docs, vector_size=200, window=3, alpha=0.05, min_count=1, seed=42, workers=1, epochs=20)
tst_model.save("test_model")

In [68]:
training_data["vector"] =""
for i in range(12145):
    training_data["vector"].iloc[i] = tst_model[i]
training_data

,filename,query,project,vector,doc_vector
0,arraystack.java,licens apach softwar foundat A S F) contributo...,COLLECTIONS,"[0.6373648, 0.36343274, -0.21556364, 0.0261671...",
1,bag.java,licens apach softwar foundat A S F) contributo...,COLLECTIONS,"[0.39248475, 0.9095622, -0.31791908, -0.447419...",
2,bagutils.java,licens apach softwar foundat A S F) contributo...,COLLECTIONS,"[-0.3806162, 0.014073659, 0.83630836, 0.206107...",
3,beanmap.java,licens apach softwar foundat A S F) contributo...,COLLECTIONS,"[-0.018915145, -0.20437856, 0.5265517, -0.2119...",
4,bidimap.java,licens apach softwar foundat A S F) contributo...,COLLECTIONS,"[0.4636378, 0.3857351, -0.26127335, -0.1524405...",
...,...,...,...,...,...
12140,bug,undertow H T T P S listen offer cipher suit D ...,ELY,"[0.5873729, -0.26671422, 0.26491788, 0.1084738...",
12141,bug,miss null check equals() method abstract permi...,ELY,"[-0.41595843, -0.3777295, 0.15903723, -0.05837...",
12142,bug,No log messag com elytron group assignmentelyt...,ELY,"[0.10762189, 0.04025764, 0.188332, 0.17249116,...",
12143,bug,elytron introduc S S L T L S protocol constrai...,ELY,"[-0.066642396, -0.86785316, 0.024531504, 0.084...",


In [46]:
# all of the vectors have 200 dimensions
print(len(training_data["vector"].iloc[0]))
print(len(training_data["vector"].iloc[1]))
print(len(training_data["vector"].iloc[2]))
print(len(training_data["vector"].iloc[12144]))

200
200
200
200


In [152]:
# testing some of the similarities and the .infer_vector function
from sklearn.metrics.pairwise import cosine_similarity
cosSim = cosine_similarity(training_data["vector"].iloc[0].reshape(1,-1), training_data["vector"].iloc[12144].reshape(1,-1)).flatten()
display(cosSim)
cosSim2 = cosine_similarity(training_data["vector"].iloc[0].reshape(1,-1), training_data["vector"].iloc[12143].reshape(1,-1)).flatten()
display(cosSim2)
tst_vector = tst_model.infer_vector(training_data["query"].iloc[12143].split())


display(training_data["vector"].shape)
lst_vects = training_data["vector"].tolist()
display(len(lst_vects))
print("list")
simList = cosine_similarity(training_data['vector'].iloc[12143].reshape(1,-1), lst_vects).flatten()
display(simList)
print(len(simList))

array([0.16581872], dtype=float32)

array([-0.07493816], dtype=float32)

(12145,)

12145

list


array([-0.07493816, -0.11288125, -0.19095442, ...,  0.01994835,
        0.9999998 ,  0.01686892], dtype=float32)

12145


### Add the vector column to the orginal data frames (sc_df & br_df)

In [117]:
sc_df["vector"] = training_data.loc[0:sc_df.shape[0], 'vector']

t_bugs = training_data.iloc[sc_df.shape[0]:training_data.shape[0]].copy()
vect_list = t_bugs.vector.tolist()
br_df['vector'] = vect_list


### Group the data again, perform same functions as before

In [129]:
# get a list of the projects 
projects = sc_df.project.unique()

# group the data frames
sc_grouped_df = sc_df.groupby(sc_df.project)
bg_grouped_df = br_df.groupby(br_df.project)

### Run slightly different version of method 2 code to generate similarity scores
- We don't have to train and run a vectorizer. 
- Just have to iterate through the bugs in the project and generate similarity scores between it's vector and the vector for each of the source code files (direct and indirect)
- For each query we want to have an array of similarity scores where each item is for a source code file

### Calculate Direct and Indirect scores

In [124]:
# code for determining the number of terms in a source code file

# min max scaler 
def custom_min_max(arr):
    min_val = np.amin(arr)
    max_val = np.amax(arr)
    f = lambda x: (x - min_val) / (max_val - min_val)
    result = f(arr)
        
    return result

# generate number of terms based off of length of file 
def gen_num_terms(len_arr):
    len_norm = custom_min_max(len_arr)
    f = lambda x: 1 / (1 + np.exp(-1 * x))
    num_terms = f(len_norm)
    return num_terms
    

In [184]:
# calculate the similarity when using revised Vector Space Model
def calculate_rVSM_similarity(src_vect, query, num_terms):
    
    cosSim = cosine_similarity(query.reshape(1,-1), src_vect).flatten()
    for i in range(len(cosSim)):
        cosSim[i] = cosSim[i] * num_terms[i]
    return cosSim

# calculate the similarity when using basic cosine similarity, used in indirect similarity calculation   
def calculate_similarity(src_vect, query):    
    cosSim = cosine_similarity(query.reshape(1,-1), src_vect).flatten()
    return cosSim


# generates direct and indirect scores
# source - dataframe for source code files
# query - dataframe for 
def generate_scores_list(source, query):
    direct_scores = []
    indirect_scores = []
    
    # create hash lookup table to decrease search time for filename index.
    lookup_table = dict()
    names = source.filename.tolist()
    for i in range(len(names)):
        lookup_table[names[i]] = i
    
    # used to define the number of terms for each source code file
    source_lengths = source['unprocessed_code'].map(lambda x: len(x.split()))
    
    # get the number of terms for each file
    num_terms_list = gen_num_terms(source_lengths)
    
    # get the DIRECT and INDIRECT similarity scores for the bug reports
    src_vects = source['vector'].tolist()
    src_code_len = len(source['unprocessed_code'])
    prev_bug_fixes = query["fix"].tolist()
    num_fixes = query["numFixes"].tolist()
    prev_vects = query['vector'].tolist()
    for q in prev_vects:
        # calculate direct similarity and append it to the list
        similarity = calculate_rVSM_similarity(src_vects, q, num_terms_list)
        direct_scores.append(similarity)
        print("Its length: ", len(similarity))
        print("Min: ", min(similarity))
        print("Max: ", max(similarity))
        # calculate indirect similarity and append it to the list
        
        indirect_similarity = calculate_indirect_scores(src_code_len, q, prev_vects, 
                                                        prev_bug_fixes, num_fixes, lookup_table)
        indirect_scores.append(indirect_similarity)

    print(len(direct_scores))
    print(len(indirect_scores))
    return direct_scores, indirect_scores

# caclulate the similarity between new bugs and old bugs.
def calculate_indirect_scores(src_len, query_vect, prev_vects, prev_bugs, num_fixes, table):
    sim_1 = 0
    # np array of zeros, update the values as needed.
    sim_scores = np.zeros(src_len)
    
    # CAN'T COMPARE A BUG TO IT'S SELF
    # get similarity between the query and the prev bug query
    bugs_sim = calculate_similarity(prev_vects, query_vect) # one of the entries should be 1
    print("Bug Similarity: ", bugs_sim)
    num_bugs = len(prev_bugs)
    print("The number of bugs: ", num_bugs)
    print("The number of sim scores: ", len(bugs_sim))
    for indx in range(num_bugs):
        
#         print("Index: ", indx)

    # get the number of fixes, used for calculating similarity
        num_fix = num_fixes[indx]
        
        # for each fix find it's index in the source['filename'] column
        for fix_indx in range(num_fix):

            sim_indx = table.get(prev_bugs[indx][fix_indx])
            if(sim_indx):
                if(bugs_sim[indx] == 1):
                    # don't add the similarity values if they are for the same bug
                    print("There was a similarity of 1")
                    sim_1 +=1
                    pass
                else:
                    sim_scores[sim_indx] = sim_scores[sim_indx] + (bugs_sim[indx]/num_fix)
#             else:
#                 missing_count += 1
    # now we have a list of indirect similarity scores for a single bug and all src code files
    print("The number of similarities = 1: ", sim_1)
    return sim_scores

    
    

In [185]:
bugs, sources = generate_all_scores()

Getting scores for project  COLLECTIONS ...
Its length:  476
Min:  -0.13297288
Max:  0.19903174
Bug Similarity:  [ 1.          0.1948739   0.32221705  0.21672751  0.3426268   0.48177105
  0.56366193  0.22483498  0.2647318   0.45557106  0.4413628   0.50261474
  0.37282076  0.47014135  0.3876177   0.18215397  0.34576288  0.3003171
  0.45088422  0.3320157   0.4997428   0.28188992  0.53278446  0.45022616
  0.5498046   0.31772017  0.47659555  0.39748794  0.2539854   0.2231869
  0.1555391   0.08224642  0.1065862  -0.00341382  0.17793868  0.07180794
  0.10106266  0.07350092 -0.14882901  0.01579405  0.25832844  0.1609796
 -0.08057263  0.28416085  0.41433015  0.12060823  0.20137322  0.2874258
  0.11458115  0.2966026   0.38424054  0.19253236  0.06041458  0.26174387
  0.17556766  0.05893552  0.18105239  0.38224423  0.33242676  0.5448995
  0.26095244  0.38299876  0.10871134  0.3029282   0.20194912  0.32637867
  0.1424763   0.10684193  0.4796516   0.02500472  0.16344571  0.1054763
  0.09153333  0.1

Its length:  476
Min:  -0.096855864
Max:  0.23908348
Bug Similarity:  [ 0.45557106  0.44196546  0.72569704  0.23856255  0.41170132  0.75672615
  0.5099995   0.5368366   0.10922323  1.          0.03824587  0.56048214
  0.13759036  0.13644463  0.02625012  0.17454173  0.7065388   0.03763098
  0.17344508  0.2897715   0.11478946  0.29312983  0.4062041   0.42328703
  0.3814137   0.48163795  0.38070118  0.62690645  0.56649125  0.5882368
  0.08336876  0.12553848  0.14199479  0.02177355 -0.06291863  0.0635779
 -0.08928813  0.1803975   0.02875363  0.28361946  0.13518208  0.02866344
  0.23519559  0.23074561  0.20730293  0.18748301 -0.00910475  0.40593576
  0.5893686   0.20406657  0.22221114  0.07347344  0.29365772  0.2706535
  0.21133743  0.22544077  0.1988449   0.21773775  0.28239268  0.45264873
  0.42068815  0.39698854  0.4199429   0.18318395  0.20268951  0.1268382
  0.2696819   0.44639295  0.24503605  0.03884888 -0.0805933   0.1136451
  0.33605748  0.1959832   0.26239413  0.34954387  0.2731406

Its length:  476
Min:  -0.15043855
Max:  0.24684459
Bug Similarity:  [ 0.3320157   0.2859776   0.36690575  0.01479299  0.24441658  0.3532346
  0.31266814  0.40539876  0.29793066  0.2897715   0.1913195   0.42653024
  0.09342927  0.36592287  0.39972097  0.461654    0.27467936  0.4704375
  0.48623872  1.          0.48225644 -0.0445494   0.2204719   0.23882866
  0.39476377  0.49259743  0.28624117  0.14965476 -0.03347405  0.03864611
 -0.06255872 -0.22622935  0.00356169  0.03847869  0.18422113  0.01172952
  0.04856042  0.09406848 -0.12314683  0.11609828  0.04795645  0.1822746
  0.20015058  0.26963082  0.25192317  0.00825565 -0.01428103  0.29298282
  0.18905307  0.06908295  0.17098904  0.17966808 -0.08775146  0.06885029
  0.00444204 -0.01369564  0.35217732  0.10108233  0.16440271  0.42997026
  0.17258754  0.07837369  0.2119026   0.07844992  0.09629188  0.20627615
  0.21491545  0.15479106  0.55510104 -0.0019383  -0.02426494  0.05435138
  0.06044645  0.34383017  0.06575318  0.19966951  0.272081

Its length:  476
Min:  -0.13831985
Max:  0.18545644
Bug Similarity:  [ 1.55539095e-01  7.16870725e-02  1.00142896e-01  6.50521740e-02
  1.28566176e-01 -3.22807468e-02  4.30071801e-02  9.25163999e-02
  5.96298575e-01  8.33687633e-02  2.62291402e-01  1.37023419e-01
 -1.06071755e-01  2.74049640e-01  2.59225190e-01  4.91199270e-02
  2.84985788e-02  1.29252240e-01  1.50836766e-01 -6.25587180e-02
  2.07617953e-01  2.06874937e-01  1.80019200e-01  1.93093091e-01
  2.38017991e-01 -7.81604648e-02  2.88219750e-01  2.82898396e-02
  1.46249115e-01  7.32012242e-02  1.00000000e+00  4.09462184e-01
  1.97445631e-01  2.73551524e-01  7.68539011e-02 -3.64213735e-02
  1.42966583e-01  4.03893977e-01  5.58367744e-02 -1.28175914e-01
  1.97256655e-01 -2.74689421e-02 -2.93582827e-02  6.16164923e-01
  2.24304050e-01  1.59742400e-01  1.83142394e-01  9.44699869e-02
 -1.49155706e-01  1.70693129e-01  2.89132386e-01  1.15209214e-01
  3.38433146e-01  5.64292222e-02  1.36981279e-01  1.16322592e-01
  1.20053127e-01  2.2

Bug Similarity:  [ 0.1609796   0.1960505   0.10665879  0.38795865  0.2711489   0.13998848
  0.21171089  0.09120857  0.24260348  0.02866344  0.3110483   0.15512556
  0.17831135  0.10647515  0.30619404  0.49129844  0.01630003  0.273203
  0.37906498  0.1822746   0.17642567  0.05666986  0.0039311   0.09428337
  0.11345463  0.08731496  0.11364224  0.06779235 -0.00925383  0.05309218
 -0.02746894  0.0649734  -0.23291679 -0.04503984  0.23404956  0.1892302
  0.14362417  0.45036235 -0.01672292  0.5922444  -0.09313644  1.
  0.01035929  0.14831653  0.28672794  0.43127248  0.3449723  -0.06105949
  0.05374392  0.12467493  0.19021887  0.17759204 -0.22490391  0.20076564
  0.14405833  0.03718199  0.15481982  0.23751777  0.24337265  0.23694204
  0.16107476  0.2078081   0.15759698  0.06133099  0.05827644  0.14877947
  0.08849658  0.11362861  0.24069566 -0.05602484  0.35207218  0.16242167
  0.06545672  0.22232729 -0.25384396  0.24867544 -0.0493423   0.0381136
  0.06201968  0.23464355  0.12227511  0.166950

Its length:  476
Min:  -0.14663368
Max:  0.18165992
Bug Similarity:  [ 1.92532361e-01 -1.71561316e-01 -1.80022232e-02  3.53483379e-01
  5.52142859e-02  6.50218427e-02  8.83350000e-02 -7.95140862e-03
  8.40727761e-02  7.34734386e-02  1.46844208e-01  1.65513963e-01
  1.87005639e-01  3.68256807e-01  1.07369393e-01 -1.68800920e-01
 -7.86024779e-02  5.27339578e-02  2.73403674e-02  1.79668084e-01
  1.18050717e-01  1.58040076e-01  2.10980415e-01 -2.78929900e-02
  1.03481188e-01 -1.08587958e-01  7.13957250e-02  9.02947783e-02
 -1.41267432e-02 -1.11530814e-02  1.15209214e-01  2.97672391e-01
 -7.00535253e-02  2.81011403e-01  1.08755268e-02 -2.13500530e-01
  2.25725204e-01  8.54991972e-02  1.16386995e-01  6.39016256e-02
  6.40567467e-02  1.77592039e-01 -1.69702262e-01  1.04508631e-01
 -1.89190894e-01  6.17779791e-02  1.64566204e-01  1.00302324e-02
 -1.61430016e-01  1.27308860e-01  1.58501193e-02  9.99999881e-01
 -1.96947277e-01 -4.10672836e-02  6.75826818e-02 -3.09341908e-01
  7.63905421e-03 -2.0

 0.23859662]
The number of bugs:  85
The number of sim scores:  85
The number of similarities = 1:  0
Its length:  476
Min:  -0.02482741
Max:  0.3266615
Bug Similarity:  [ 0.10871134  0.578333    0.6097535   0.24780425  0.610895    0.46513888
  0.48570818  0.61869353  0.19148186  0.4199429   0.2842633   0.35270154
  0.06278987  0.01494003  0.22488733  0.29908472  0.48574483  0.21674535
  0.19975314  0.2119026   0.18050498  0.04094152  0.08355144  0.13105883
  0.15304434  0.5421504  -0.01599042  0.42637765  0.4689692   0.57144415
  0.04487977 -0.04621284  0.06039263  0.09218068  0.29557106  0.22953553
  0.04349197  0.32530668  0.13179699  0.4645795   0.12075841  0.15759698
  0.4678933   0.30062515  0.4313464   0.14045084  0.20552978  0.56783986
  0.580449    0.23304003  0.17227095 -0.16160436  0.4046207   0.41561878
  0.21596742  0.5110035   0.17566067  0.39654174  0.01721741  0.12491235
  0.4869039   0.31555605  1.          0.15344591  0.14508522 -0.04418274
  0.587744    0.42325032  0

The number of similarities = 1:  1
Its length:  476
Min:  -0.16675113
Max:  0.21981673
Bug Similarity:  [ 1.63445711e-01  2.52477303e-02 -1.01990998e-02  6.92021698e-02
  2.06990764e-01  2.21635737e-02  2.11811393e-01  1.50514603e-01
  1.39715716e-01 -8.05933028e-02  5.48846364e-01  2.22347885e-01
  9.55054536e-04  3.55505913e-01  2.06048042e-01  1.78659484e-02
 -1.54848360e-02  1.34291932e-01  1.50859073e-01 -2.42649391e-02
  3.62099826e-01 -6.95785210e-02 -1.29347339e-01 -1.46384835e-01
 -3.32748741e-02  5.77670187e-02 -1.24960236e-01  2.28158310e-01
  1.06869273e-01  1.86708659e-01  1.17381796e-01 -3.86303291e-02
  3.93227972e-02 -3.26461866e-02  3.09487104e-01  1.38733163e-01
  1.05474293e-01  1.99645504e-01 -8.66246000e-02  3.01753223e-01
  1.35488003e-01  3.52072179e-01 -3.59185413e-02  9.63579044e-02
  1.55441061e-01  1.94401652e-01  1.06952392e-01  7.25719333e-02
  2.08553374e-02  1.63334519e-01  2.98655123e-01  4.89361919e-02
  6.60206825e-02  2.15595037e-01 -4.90103140e-02  1

Its length:  476
Min:  -0.10015338
Max:  0.21541107
Bug Similarity:  [ 0.11043404  0.10103067  0.13531938  0.08989216  0.15059574  0.06778048
  0.05311874  0.13829368  0.19198896 -0.10995722  0.19981097  0.13514602
  0.19682744  0.40722418  0.33297265  0.08242975  0.01447771  0.16057724
  0.17848173  0.2713074   0.2867621   0.0458061   0.23536111  0.07805273
  0.16756585 -0.02320978  0.14127026  0.05930885  0.04364077  0.05110898
  0.30197364  0.00252989  0.0021906   0.08504265  0.4209052  -0.00295621
  0.20210853  0.08477669 -0.0271943   0.00863555  0.39111173  0.06201968
 -0.00235269  0.23495105  0.11078596 -0.08991896  0.13848819  0.3031382
 -0.1252723   0.00532447  0.09939731  0.35762733  0.08684841 -0.00232407
 -0.07068723  0.09741599  0.11075784 -0.07942609  0.07036069  0.10698524
  0.25084144  0.14261003  0.0789158  -0.00686501  0.04359741 -0.03437836
  0.3835375   0.18043628  0.2924646  -0.19059025  0.10819432 -0.09972946
  0.02786426 -0.09250005 -0.10221486  0.02663032  0.2561

### Rank the similarity scores and Compute MAP and MRR

In [126]:
# rank all the similarity scores
def rank_sim_scores(scores):
    sim_scores = list()
    
    for score in scores:
        indicies = range(len(score))

        scores_tuple = tuple(zip(score,indicies))
        sorted_tuple = sorted(scores_tuple, reverse = True)

        sim_scores.append(sorted_tuple)
    
    return sim_scores

#Checks the average precision for each bug
def average_precision(fix_indexes,ranked_sim):
    ap_list = list()
    for fixes,ranked_list in zip(fix_indexes,ranked_sim):
        hit_list = list()
        countTrue=0
        for i in range(len(ranked_list)):
            # check if source file is actually where bug is located
            if(ranked_list[i][1] in fixes):
                countTrue+=1
                hit_list.append(countTrue/(i+1))
        if(countTrue != 0):
            ap_list.append(sum(hit_list)/countTrue)
        else:
            ap_list.append(0)
    return ap_list


#reciprocal rank is 1/n, where n is the first position of a source file where the bug is located in the ranked_sim column
def reciprocal_rank(fix_indexes,ranked_sim):
    rr_list = list()
    for fixes,ranked_list in zip(fix_indexes,ranked_sim):
        rr = 0
        for i in range(len(ranked_list)):
            # check if source file is actually where bug is located
            if(ranked_list[i][1] in fixes):
                rr = 1/(i+1)
                break
        rr_list.append(rr)
    return rr_list

# Gets a list containing the rank of all fixes that were found in the ranked similarity list
def get_fix_rank(bug, isCosineSim=True):
    fix_list = list()
    ranked_sim = 'ranked_sim'
    if not isCosineSim:
        ranked_sim = 'ranked_eq7_sim'
    for index, row in bug.iterrows():
        i_list = list()
        for i in range(len(row[ranked_sim])):
            if(row[ranked_sim][i][1] in row['fix_indexes']):
                i_list.append(i+1)
        fix_list.append(i_list)
    return fix_list

### Get metrics into the data frames

In [132]:
# take in the source code df for a project and a single query return scores
import warnings
warnings.filterwarnings("ignore")

projects2 = projects[:1]

def generate_all_scores():
    
    all_bugs = []
    all_src = []
    # iterate through the list of 12 projects
    for proj in projects2:
        print("Getting scores for project ",proj,"...")
        # create dataframes for each project
        src_df = sc_grouped_df.get_group(proj)
        bug_df = bg_grouped_df.get_group(proj).copy()
        
        # generate the direct and indirect scores
        direct_scores, indirect_scores = generate_scores_list(src_df, bug_df)
        
        #append direct scores list to the bug dataframe
        bug_df["direct_sim"] = direct_scores # the only way that the matrix is related to the src code 
                                        # is through the index.
            
        #append indirect scores list to bug dataframe
        bug_df["indirect_sim"] = indirect_scores
        
        
        bug_df["fix_indexes"] = get_fix_indexes(bug_df, src_df)

        
      
        # maintain a list of all the dataframes
        all_bugs.append(bug_df)
        all_src.append(src_df)
    # concatenate all the data frames in order    
    all_bug_df = pd.concat(all_bugs, ignore_index=True)
    all_src_df = pd.concat(all_src, ignore_index=True)
    return all_bug_df, all_src_df